# Data Consolidattion 

In [25]:
import os
import glob
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
#!pip install geopandas
#!pip install geopy
from geopy.geocoders import Nominatim
import re

In [2]:
extension = 'csv'
all_filenames = [i for i in glob.glob('*.{}'.format(extension))]

In [3]:
filenames_2019 = all_filenames[:6]

filenames_2020_2021 = all_filenames[6:]

In [4]:
#combine all files in the list
df_19 = pd.concat([pd.read_csv(f) for f in filenames_2019 ])

df_20_21 = pd.concat([pd.read_csv(f) for f in filenames_2020_2021 ])

In [5]:
# Create 4 empty columns in 2019 data

df_19['start_lat'] = " "
df_19['start_lng'] = " "
df_19['end_lat'] = " "
df_19['end_lng'] = " "

df_19['ride_id'] = " "
df_19['rideable_type'] = " "


In [6]:
# Create functions for geocoding 

def extract_lat(address):
    try:
        location = geolocator.geocode(address)
        return location.latitude
    except:
        return ''

    
def extract_long(address):
    try:
        location = geolocator.geocode(address)
        return location.longitude
    except:
        return ''

In [7]:
from geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent="example app")

In [12]:
# run in shell the next code to activate parrallel processing with all cores 

# ipcluster start -n 8

In [13]:
'''
from ipyparallel import Client
rc = Client()
'''

'\nfrom ipyparallel import Client\nrc = Client()\n'

In [14]:
'''
# check cores 
rc.ids

#To run parallel add it before code 

%%px
'''

'\n# check cores \nrc.ids\n\n#To run parallel add it before code \n\n%%px\n'

In [16]:
'''
df_19['start_lat'] = df_19['Start station'].map(lambda x: extract_long(x))

df_19['start_lng'] = df_19['Start station'].map(lambda x: extract_lat(x))


df_19['end_lat'] = df_19['End station'].map(lambda x: extract_long(x))

df_19['end_lng'] = df_19['End station'].map(lambda x: extract_lat(x))
'''

"\ndf_19['start_lat'] = df_19['Start station'].map(lambda x: extract_long(x))\n\ndf_19['start_lng'] = df_19['Start station'].map(lambda x: extract_lat(x))\n\n\ndf_19['end_lat'] = df_19['End station'].map(lambda x: extract_long(x))\n\ndf_19['end_lng'] = df_19['End station'].map(lambda x: extract_lat(x))\n"

In [9]:
import multiprocessing

multiprocessing.cpu_count() # or os.cpu_count()

16

### Make the same columns for all datasets 

In [24]:
df_19 = df_19.loc[:, ["ride_id","rideable_type","Start date","End date", "Start station", "Start station number"
           ,"End station", "End station number",'start_lat', 'start_lng', 'end_lat', 'end_lng','Member type','Bike number','Duration']]


df_19.columns = ['ride_id', 'rideable_type', 'started_at', 'ended_at','start_station_name', 'start_station_id', 
                 'end_station_name','end_station_id', 'start_lat', 'start_lng', 'end_lat', 'end_lng','member_casual', 'Bike number', 'Duration']


In [26]:
# Add columns in 2020_2021 data 

df_20_21['Bike number']= ""

#convert columns to datetime
df_20_21[['started_at','ended_at']] = df_20_21[['started_at','ended_at']].apply(pd.to_datetime)

#calculate difference between dates
df_20_21['Duration'] = (df_20_21['ended_at'] - df_20_21['started_at']) / np.timedelta64(1, 's')


In [41]:
# Change the wording in order to be aligned

df_19["member_casual"] = df_19["member_casual"].map(lambda x: "member" if x=="Member" else "casual")

### Consolidate all data 

In [43]:
df = pd.concat([df_19, df_20_21])

In [69]:
# Add semi annual column 

df[['started_at','ended_at']] = df[['started_at','ended_at']].apply(pd.to_datetime)

df['semi_annual'] = df['started_at'].map(lambda x:"H2 2019" if x.year==2019 else ("H2 2020" if x.year==2020 else "H2 2021"))


In [71]:
#export to csv
df.to_csv( "data.csv", index=False)